In [56]:
import turicreate as tc

In [57]:
from bs4 import BeautifulSoup

In [58]:
import requests

In [59]:
import cloudscraper

In [60]:
scraper = cloudscraper.create_scraper()

In [61]:
base_url = 'https://www.hemnet.se/salda/bostader?location_ids%5B%5D=473386'

In [62]:
base_page = scraper.get(base_url)

### get how many pages

In [63]:
soup = BeautifulSoup(base_page.text, 'html.parser')

In [64]:
soup.title.text

'Slutpriser bostäder - Lilla Essingen, Stockholms kommun - Hemnet'

In [65]:
container = soup.find('div', attrs={'class':'pagination'})

In [66]:
container

<div aria-label="Paginering" class="pagination" role="navigation"> <em class="current button">1</em> <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=2" rel="next">2</a> <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=3">3</a> <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=4">4</a> <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=5">5</a> <a class="next_page button button--primary" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=2" rel="next">Nästa</a></div>

In [67]:
buttons = container.find_all('a', attrs={'class':'button'})

In [68]:
buttons

[<a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=2" rel="next">2</a>,
 <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=3">3</a>,
 <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=4">4</a>,
 <a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=5">5</a>,
 <a class="next_page button button--primary" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=2" rel="next">Nästa</a>]

In [70]:
buttons[-2]

<a class="button" href="/salda/bostader?location_ids%5B%5D=473386&amp;page=5">5</a>

In [71]:
int(buttons[-2].text)

5

In [81]:
page_size = int(buttons[-2].text)

In [170]:
titles = []
addresses = []
final_prices = []
price_per_squaremeters = []
demanded_prices = []
room_numbers = []
areas = []
monthly_fees = []
operating_costs = []
year_of_constructions = []
urls = []

In [172]:
for n in range(1,page_size+1):
    page_url = base_url + '&page=' + str(n)
    print(page_url)
    
    page = scraper.get(page_url)
    page_soup = BeautifulSoup(page.text, 'html.parser')
        
    cont = page_soup.find(id='search-results')
        
    home_urls = cont.find_all('a', attrs={'class':'item-link-container'})
    
    # print(home_urls)
    
    for home_url in home_urls:
        url = home_url.attrs["href"]
        house_page = scraper.get(url)
        house_soup = BeautifulSoup(house_page.text, 'html.parser')
        
        print(url)
        
        urls.append(url)
        titles.append(house_soup.title.text)
        
        address_container = house_soup.find('h1')
        addresses.append(address_container.text.split('\n')[2].strip() )
        final_prices.append( int(house_soup.find('span', attrs={'class':'sold-property__price-value'}).text.replace(u'\xa0', u'')[:-2]) )
        
        container = house_soup.find('dl', attrs={'class':'sold-property__price-stats'})
        dds = container.find_all('dd')
        
        try:
            price_per_squaremeters.append( int(dds[0].text.replace(u'\xa0', u'')[:-5]) )
        except:
            price_per_squaremeters.append(0)
            
        try:
            demanded_prices.append( int(dds[1].text.replace(u'\xa0', u'')[:-3]) )
        except:
            demanded_prices.append(0)
        
        container = house_soup.find('dl', attrs={'class':'sold-property__attributes'})
        dds = container.find_all('dd')
        
        try:
            room_numbers.append( dds[0].text )
        except:
            room_numbers.append('')
            
        try:
            areas.append(int(dds[1].text[:-2]))
        except:
            areas.append(0)
        
        try:
            monthly_fees.append(int(dds[2].text.replace(u'\xa0', u'')[:-6]))
        except:
            monthly_fees.append(0)
        
        try:
            if dds[3].text.replace(u'\xa0', u'')[:-5] == '':
                operating_costs.append(0)
            else:
                operating_costs.append(int(dds[3].text.replace(u'\xa0', u'')[:-5]))
        except:
            operating_costs.append(0)
        
        try:
            year_of_constructions.append(int(dds[4].text))
        except:
            year_of_constructions.append(0)        

https://www.hemnet.se/salda/bostader?location_ids%5B%5D=473386&page=1
https://www.hemnet.se/salda/lagenhet-1rum-lilla-essingen-kungsholmen-stockholms-kommun-luxgatan-7-1313676
https://www.hemnet.se/salda/lagenhet-4rum-kungsholmen-lilla-essingen-stockholms-kommun-primusgatan-108,-6-6-tr-1312380
https://www.hemnet.se/salda/lagenhet-2rum-lilla-essingen-kungsholmen-stockholms-kommun-stralgatan-47,-3-tr-1312152
https://www.hemnet.se/salda/lagenhet-2,5rum-lilla-essingen-stockholms-kommun-disponentgatan-1-1312061
https://www.hemnet.se/salda/lagenhet-2rum-kungsholmen-lilla-essingen-stockholms-kommun-essinge-brogata-12-1311880
https://www.hemnet.se/salda/lagenhet-1rum-lilla-essingen-kungsholmen-stockholms-kommun-essinge-brogata-39,-2-tr-1305647
https://www.hemnet.se/salda/lagenhet-2rum-lilla-essingen-kungsholmen-stockholms-kommun-stralgatan-47-1302841
https://www.hemnet.se/salda/lagenhet-3rum-lilla-essingen-stockholms-kommun-patentgatan-1,-3tr-1299622
https://www.hemnet.se/salda/lagenhet-3rum-l

In [173]:
len(titles)

237

In [174]:
len(addresses)

237

In [175]:
len(final_prices)

237

In [176]:
sf = tc.SFrame({'id':[1,2,3],'val':['A','B','C']})

In [177]:
len(price_per_squaremeters)

237

In [178]:
len(demanded_prices)

237

In [179]:
len(room_numbers)

237

In [180]:
len(areas)

237

In [181]:
len(monthly_fees)

237

In [182]:
len(operating_costs)

237

In [183]:
len(year_of_constructions)

237

In [184]:
len(urls)

237

In [185]:
sf

id,val
1,A
2,B
3,C


In [186]:
frame = tc.SFrame({'url':urls, 'title':titles,'address':addresses, 'final_price': final_prices, 'price_per_squaremeter': price_per_squaremeters, 'demanded_price': demanded_prices,
                  'room_number': room_numbers, 'area': areas, 'monthly_fee': monthly_fees, 'operating_cost': operating_costs, 'year_of_construction': year_of_constructions})

In [187]:
frame

address,area,demanded_price,final_price,monthly_fee,operating_cost,price_per_squaremeter
Luxgatan 7,30,2350000,2550000,1733,6096,85000
"Primusgatan 108, 6/6 tr",101,11400000,11400000,5887,5540,112871
"Strålgatan 47, 3 tr",31,2595000,2875000,2471,3200,92742
Disponentgatan 1,46,3550000,3550000,3003,12979,77174
Essinge Brogata 12,0,2495000,2900000,2162,4200,84058
"Essinge Brogata 39, 2 tr",23,1895000,2205000,1787,3600,95870
Strålgatan 47,40,2595000,2850000,2957,4860,71250
"Patentgatan 1, 3tr",69,5995000,6400000,3778,7200,92754
"Primusgatan 82, 6tr",0,8495000,8495000,4828,7800,100533
"Primusgatan 81, vån 4",96,8495000,9250000,4793,6860,96354


In [188]:
frame.save(filename='./houses.csv',format='csv')

In [189]:
frame.show()

Materializing SFrame

In [190]:
frame.explore()

/usr/local/lib/python3.7/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,address,area,demanded_price,final_price,monthly_fee,operating_cost,price_per_squaremeter,room_number,title,url,year_of_construction
0,Luxgatan 7,30,2350000,2550000,1733,6096,85000,1 rum,"Luxgatan 7 - Såld bostadsrättslägenhet - Lilla Essingen \ Kungsholmen, Stockholm - Hemnet",https://www.hemnet.se/salda/lagenhet-1rum-lilla-essingen-kungsholmen-stockholms-kommun-luxgatan-7-1313676,1937
1,"Primusgatan 108, 6/6 tr",101,11400000,11400000,5887,5540,112871,4 rum,"Primusgatan 108, 6/6 tr - Såld bostadsrättslägenhet - Kungsholmen / Lilla Essingen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-4rum-kungsholmen-lilla-essingen-stockholms-kommun-primusgatan-108,-6-6-tr-1312380",1935
2,"Strålgatan 47, 3 tr",31,2595000,2875000,2471,3200,92742,2 rum,"Strålgatan 47, 3 tr - Såld bostadsrättslägenhet - Lilla Essingen - Kungsholmen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-2rum-lilla-essingen-kungsholmen-stockholms-kommun-stralgatan-47,-3-tr-1312152",1939
3,Disponentgatan 1,46,3550000,3550000,3003,12979,77174,"2,5 rum","Disponentgatan 1 - Såld bostadsrättslägenhet - Lilla Essingen, STOCKHOLM - Hemnet","https://www.hemnet.se/salda/lagenhet-2,5rum-lilla-essingen-stockholms-kommun-disponentgatan-1-1312061",1935
4,Essinge Brogata 12,0,2495000,2900000,2162,4200,84058,2 rum,"Essinge Brogata 12 - Såld bostadsrättslägenhet - Kungsholmen - Lilla Essingen, Stockholm - Hemnet",https://www.hemnet.se/salda/lagenhet-2rum-kungsholmen-lilla-essingen-stockholms-kommun-essinge-brogata-12-1311880,1934
5,"Essinge Brogata 39, 2 tr",23,1895000,2205000,1787,3600,95870,1 rum,"Essinge Brogata 39, 2 tr - Såld bostadsrättslägenhet - Lilla Essingen - Kungsholmen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-1rum-lilla-essingen-kungsholmen-stockholms-kommun-essinge-brogata-39,-2-tr-1305647",1938
6,Strålgatan 47,40,2595000,2850000,2957,4860,71250,2 rum,"Strålgatan 47 - Såld bostadsrättslägenhet - Lilla Essingen \ Kungsholmen, Stockholm - Hemnet",https://www.hemnet.se/salda/lagenhet-2rum-lilla-essingen-kungsholmen-stockholms-kommun-stralgatan-47-1302841,1939
7,"Patentgatan 1, 3tr",69,5995000,6400000,3778,7200,92754,3 rum,"Patentgatan 1, 3tr - Såld bostadsrättslägenhet - Lilla Essingen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-3rum-lilla-essingen-stockholms-kommun-patentgatan-1,-3tr-1299622",2006
8,"Primusgatan 82, 6tr",0,8495000,8495000,4828,7800,100533,3 rum,"Primusgatan 82, 6tr - Såld bostadsrättslägenhet - Lilla Essingen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-3rum-lilla-essingen-stockholms-kommun-primusgatan-82,-6tr-1299073",2004
9,"Primusgatan 81, vån 4",96,8495000,9250000,4793,6860,96354,4 rum,"Primusgatan 81, vån 4 - Såld bostadsrättslägenhet - Lilla Essingen \ Kungsholmen, Stockholm - Hemnet","https://www.hemnet.se/salda/lagenhet-4rum-lilla-essingen-kungsholmen-stockholms-kommun-primusgatan-81,-van-4-1298651",2004
